In [ ]:
import pandas as pd
ratings_data = pd.read_csv('./podaci/rating.csv')
movies_data = pd.read_csv('./podaci/movie.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Primjer DataFrame-a sa žanrovima filmova
movies_data = pd.DataFrame({
    'movieId': [1, 2, 3],
    'title': ['Movie A', 'Movie B', 'Movie C'],
    'genres': ['Action|Adventure|Sci-Fi', 'Comedy|Drama', 'Action|Drama|Thriller']
})

# Primjer DataFrame-a sa ocjenama korisnika
ratings_data = pd.DataFrame({
    'userId': [1, 2, 3],
    'movieId': [1, 2, 3],
    'rating': [5, 4, 3]
})

# Novi film koji želimo predviđati
new_movie = pd.DataFrame({
    'movieId': [4],
    'title': ['New Movie'],
    'genres': ['Action|Adventure']
})

# Funkcija za izračunavanje sličnosti između novog filma i postojećih filmova
def predict_average_rating(new_movie, movies_data, ratings_data):
    # Dodajemo novi film u skup podataka
    movies_data = pd.concat([movies_data, new_movie])

    # Stvaranje matrice ocjena
    user_movie_matrix = pd.pivot_table(ratings_data, values='rating', index='userId', columns='movieId', fill_value=0)

    # Računanje sličnosti između filmova pomoću kosinusne sličnosti
    movie_similarity = cosine_similarity(user_movie_matrix.T)

    # Izdvajamo vektor žanrova za novi film
    new_movie_genres = new_movie['genres'].str.get_dummies().reindex(columns=user_movie_matrix.columns, fill_value=0)

    # Računamo sličnost između novog filma i postojećih filmova
    new_movie_similarity = cosine_similarity(new_movie_genres, user_movie_matrix.T)

    # Računamo ponderiranu sličnost između novog filma i ocijenjenih filmova
    weighted_similarity = np.dot(new_movie_similarity, user_movie_matrix) / np.sum(np.abs(new_movie_similarity))

    # Računamo predviđene ocjene za novi film
    predicted_ratings = np.dot(weighted_similarity, ratings_data.set_index('movieId')['rating']) / np.sum(np.abs(weighted_similarity))
    print(predicted_ratings)

    return predicted_ratings[new_movie['movieId'].values[0]]

# Predviđanje prosječne ocjene za novi film
predicted_rating = predict_average_rating(new_movie, movies_data, ratings_data)

# Prikaz rezultata
print(f"Predicted average rating for '{new_movie['title'].values[0]}': {predicted_rating:.2f}")


: 

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Dodajte izmišljeni film za koji želite predviđati ocjene
new_movie_id = 99999

# Pretvorba podataka u DataFrame
new_movie_data = pd.DataFrame({'movieId': [99999], 'title': 'test', 'genres': 'Action|Comedy'})

# Dodavanje novih ocjena u skup podataka
movies_data = pd.concat([movies_data, new_movie_data], ignore_index=True)

# Izračun sličnosti između filmova koristeći cosine similarity
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
cosine_sim = cosine_similarity(user_movie_ratings)

# Pronalaženje najsličnijih filmova novom filmu
similar_movies = cosine_sim[0]

# Izračun težinske srednje ocjene koristeći slične filmove
weighted_sum = 0
total_weight = 0

for i, similarity in enumerate(similar_movies):
    if similarity > 0:  # Zanemarujemo neslične filmove
        weighted_sum += similarity * user_movie_ratings.iloc[:, i].mean()
        total_weight += similarity

predicted_rating = weighted_sum / total_weight if total_weight > 0 else None

print(f'Predicted average rating for the new movie: {predicted_rating}')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Funkcija za određivanje sličnosti između korisnika koristeći cosine similarity
def find_similar_users(user_id, ratings_matrix):
    user_ratings = ratings_matrix.loc[user_id].values.reshape(1, -1)
    similarity = cosine_similarity(ratings_matrix, user_ratings)
    similar_users = pd.DataFrame(similarity, index=ratings_matrix.index, columns=['Similarity']).sort_values(by='Similarity', ascending=False)
    return similar_users

# Pretvorba podataka u matricu ocjena korisnika
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Pronalaženje sličnih korisnika za određenog korisnika (primjer: korisnik s ID-om 1)
target_user_id = 1
similar_users = find_similar_users(target_user_id, user_movie_ratings)

# Ispis sličnih korisnika
print(f'Similar users for user {target_user_id}:\n{similar_users.head()}')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Funkcija za dobivanje preporuka za korisnika na temelju sličnih korisnika
def get_movie_recommendations(user_id, ratings_matrix, similar_users, n_recommendations=5):
    user_ratings = ratings_matrix.loc[user_id].values.reshape(1, -1)
    
    # Izračun sličnosti između korisnika i ocjena korisnika za filmove
    similarity = cosine_similarity(ratings_matrix, user_ratings)
    
    # Uzimanje težinskih ocjena korisnika za filmove sličnih korisnika
    weighted_ratings = similarity.T @ ratings_matrix.values
    
    # Normalizacija težinskih ocjena
    normalized_ratings = weighted_ratings / (1e-10 + abs(similarity).sum(axis=0))
    
    # Pronalaženje filmova koje ciljni korisnik još nije ocijenio
    unrated_movies = ratings_matrix.loc[user_id][ratings_matrix.loc[user_id].isna()].index
    
    # Sortiranje preporučenih filmova prema ocjenama
    recommendations = pd.DataFrame(normalized_ratings[:, unrated_movies], index=ratings_matrix.index, columns=unrated_movies)
    top_recommendations = recommendations.sum().sort_values(ascending=False).head(n_recommendations)
    
    return top_recommendations

# Pretvorba podataka u matricu ocjena korisnika
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Pronalaženje sličnih korisnika za određenog korisnika (primjer: korisnik s ID-om 1)
target_user_id = 1
similar_users = find_similar_users(target_user_id, user_movie_ratings)

# Dobivanje preporuka za ciljnog korisnika
recommendations = get_movie_recommendations(target_user_id, user_movie_ratings, similar_users)
print(f'Recommended movies for user {target_user_id}:\n{recommendations}')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Funkcija za predviđanje ocjene korisnika za određeni film
def predict_rating(user_id, movie_id, ratings_matrix, movie_similarity):
    user_ratings = ratings_matrix.loc[user_id]
    similar_movies = movie_similarity[movie_id].sort_values(ascending=False).index
    
    # Filtriranje sličnih filmova koje korisnik već ocijenio
    similar_movies_rated = [movie for movie in similar_movies if not pd.isna(user_ratings[movie])]
    
    # Ako nema sličnih ocijenjenih filmova, vraćamo None
    if len(similar_movies_rated) == 0:
        return None
    
    # Izračun ponderirane ocjene temeljene na sličnosti filmova
    weighted_sum = sum(user_ratings[movie] * movie_similarity[movie_id][movie] for movie in similar_movies_rated)
    total_similarity = sum(abs(movie_similarity[movie_id][movie]) for movie in similar_movies_rated)
    
    predicted_rating = weighted_sum / total_similarity
    return predicted_rating

# Pretvorba podataka u matricu ocjena korisnika
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Izračun sličnosti između filmova koristeći cosine similarity
movie_similarity = cosine_similarity(user_movie_ratings.T)

# Primjer predviđanja ocjene za korisnika s ID-om 1 i film s ID-om 1001
target_user_id = 1
target_movie_id = 1001
predicted_rating = predict_rating(target_user_id, target_movie_id, user_movie_ratings, movie_similarity)

print(f'Predicted rating for user {target_user_id} and movie {target_movie_id}: {predicted_rating}')

In [ ]:
# Spojite podatke o ocjenama i oznakama
merged_data = pd.merge(ratings_data, tags_data, on='movieId', how='left')
positive_tags = ['action', 'adventure', 'comedy', 'romance']  # Prilagodite popis oznaka prema vašim potrebama
positive_movies = merged_data[merged_data['tag'].isin(positive_tags)]
train_data, test_data = train_test_split(positive_movies, test_size=0.2, random_state=42)
# Primjer: collaborative filtering model
from sklearn.metrics.pairwise import cosine_similarity

user_movie_ratings = train_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
movie_similarity = cosine_similarity(user_movie_ratings.T)

# Implementirajte funkciju za dobivanje preporuka na temelju sličnosti
def get_recommendations(movie_id, movie_similarity, n_recommendations=5):
    similar_movies = pd.Series(movie_similarity[movie_id]).sort_values(ascending=False)
    similar_movies = similar_movies[similar_movies.index != movie_id]  # Isključi sam film
    top_recommendations = similar_movies.head(n_recommendations)
    return top_recommendations

# Primjer korištenja: Preporuke za film s ID-om 123
movie_id_to_recommend = 123
recommendations = get_recommendations(movie_id_to_recommend, movie_similarity)
print(f'Recommended movies for movie {movie_id_to_recommend}:\n{recommendations}')

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split

# Pretpostavljeni DataFrame s ocjenama
# ratings_data = ...

# Pretpostavljeni DataFrame s oznakama za filmove
# tags_data = ...

# Pretpostavljeni DataFrame s informacijama o filmovima
# movies_data = ...
# Spojite podatke o ocjenama i oznakama
merged_data = pd.merge(ratings_data, tags_data, on='movieId', how='left')

# Spojite s informacijama o filmovima
merged_data = pd.merge(merged_data, movies_data, on='movieId', how='left')
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_data['tags'].fillna(''))
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)
cosine_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
def get_recommendations(movie_id, cosine_similarity_matrix, n_recommendations=5):
    movie_index = merged_data[merged_data['movieId'] == movie_id].index[0]
    similarity_scores = list(enumerate(cosine_similarity_matrix[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_recommendations = similarity_scores[1:n_recommendations + 1]  # Preskoči prvi (jer je to sam film)
    recommended_movie_indices = [index for index, _ in top_recommendations]
    recommended_movies = merged_data.iloc[recommended_movie_indices]['movieId'].values
    return recommended_movies


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Pretpostavljeni DataFrame s ocjenama
# ratings_data = ...

# Pretpostavljeni DataFrame s oznakama za filmove
# tags_data = ...

# Pretpostavljeni DataFrame s informacijama o filmovima
# movies_data = ...
# Spojite podatke o ocjenama i oznakama
merged_data = pd.merge(ratings_data, tags_data, on='movieId', how='left')

# Spojite s informacijama o filmovima
merged_data = pd.merge(merged_data, movies_data, on='movieId', how='left')
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)
# Primjer: sličnost među korisnicima
from sklearn.metrics.pairwise import cosine_similarity

user_movie_ratings = train_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
user_similarity = cosine_similarity(user_movie_ratings)

# Primjer: sličnost među filmovima
movie_tags_matrix = tags_data.pivot(index='movieId', columns='tag', values='tag').fillna(0)
movie_similarity = cosine_similarity(movie_tags_matrix)
def get_recommendations(user_id, user_similarity, movie_similarity, n_recommendations=5):
    user_index = train_data[train_data['userId'] == user_id].index[0]
    
    # Pronalaženje sličnih korisnika
    similar_users = sorted(list(enumerate(user_similarity[user_index])), key=lambda x: x[1], reverse=True)[1:]
    similar_users_indices = [index for index, _ in similar_users][:5]  # Odaberi prvih 5 sličnih korisnika
    
    # Prikupljanje filmova koje su slični korisnici ocijenili
    recommended_movies = set()
    for similar_user_index in similar_users_indices:
        user_movies = train_data[train_data['userId'] == similar_user_index]['movieId']
        recommended_movies.update(user_movies)
    
    # Isključivanje filmova koje je korisnik već ocijenio
    user_rated_movies = set(train_data[train_data['userId'] == user_id]['movieId'])
    recommended_movies -= user_rated_movies
    
    # Prikupljanje top preporuka
    top_recommendations = sorted(list(recommended_movies), key=lambda x: movie_similarity[user_index][x], reverse=True)[:n_recommendations]
    return top_recommendations


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.model_selection import train_test_split

# Pretpostavljeni DataFrame s ocjenama
# ratings_data = ...

# Pretpostavljeni DataFrame s oznakama za filmove
# tags_data = ...

# Pretpostavljeni DataFrame s informacijama o filmovima
# movies_data = ...
# Spojite podatke o ocjenama i oznakama
merged_data = pd.merge(ratings_data, tags_data, on='movieId', how='left')

# Spojite s informacijama o filmovima
merged_data = pd.merge(merged_data, movies_data, on='movieId', how='left')
train_data, test_data = train_test_split(merged_data, test_size=0.2, random_state=42)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
tfidf_matrix = tfidf_vectorizer.fit_transform(merged_data['tags'].fillna(''))
cosine_similarity_matrix = linear_kernel(tfidf_matrix, tfidf_matrix)
def get_recommendations(user_id, tag, cosine_similarity_matrix, n_recommendations=5):
    user_movies_with_tag = merged_data[(merged_data['userId'] == user_id) & (merged_data['tag'] == tag)]
    if user_movies_with_tag.empty:
        return []  # Korisnik nije dodijelio tu oznaku filmovima
        
    movie_index = user_movies_with_tag.sample(1).index[0]
    similarity_scores = list(enumerate(cosine_similarity_matrix[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_recommendations = similarity_scores[1:n_recommendations + 1]  # Preskoči prvi (jer je to sam film)
    recommended_movie_indices = [index for index, _ in top_recommendations]
    recommended_movies = merged_data.iloc[recommended_movie_indices]['movieId'].values
    return recommended_movies
# Izmislite korisnika i oznaku koju je već koristio
user_id = 4
tag_used_by_user = 'action'

# Dobivanje preporuka pozivom funkcije
recommendations = get_recommendations(user_id, tag_used_by_user, cosine_similarity_matrix)

# Ispis preporuka
print(f"Recommendations for user {user_id} with tag '{tag_used_by_user}':")
for movie_id in recommendations:
    movie_title = movies_data[movies_data['movieId'] == movie_id]['title'].values[0]
    print(f" - {movie_title} (Movie ID: {movie_id})")

    def get_recommendations(user_id, tag, user_movie_tags, cosine_similarity_matrix, n_recommendations=5):
    user_movies_with_tag = user_movie_tags[(user_movie_tags['userId'] == user_id) & (user_movie_tags['tag'] == tag)]
    if user_movies_with_tag.empty:
        return []  # Korisnik nije dodijelio tu oznaku filmovima

    movie_index = user_movies_with_tag.sample(1).index[0]
    similarity_scores = list(enumerate(cosine_similarity_matrix[movie_index]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    top_recommendations = similarity_scores[1:n_recommendations + 1]  # Preskoči prvi (jer je to sam film)
    recommended_movie_indices = [index for index, _ in top_recommendations]
    recommended_movies = user_movie_tags.iloc[recommended_movie_indices]['movieId'].values
    return recommended_movies
# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['action', 'comedy', 'action', 'drama', 'action', 'romance', 'action', 'comedy']
})

# Ostatak koda za pripremu podataka i izračunavanje sličnosti ostaje nepromijenjen

# Simulacija poziva funkcije za dobivanje preporuka
user_id = 4
tag_used_by_user = 'action'
recommendations = get_recommendations(user_id, tag_used_by_user, user_movie_tags, cosine_similarity_matrix)

# Ispis preporuka
print(f"Recommendations for user {user_id} with tag '{tag_used_by_user}':")
for movie_id in recommendations:
    movie_title = movies_data[movies_data['movieId'] == movie_id]['title'].values[0]
    print(f" - {movie_title} (Movie ID: {movie_id})")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['action', 'comedy', 'action', 'drama', 'action', 'romance', 'action', 'comedy']
})

# Podijela podataka na trening i test skupove
train_data, test_data = train_test_split(user_movie_tags, test_size=0.2, random_state=42)

# Koristimo TfidfVectorizer za pretvorbu teksta oznaka u značajke
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
classifier = MultinomialNB()

# Kreiramo cjevovod koji uključuje vektorizator i klasifikator
model = Pipeline([('tfidf', tfidf_vectorizer), ('classifier', classifier)])

# Treniramo model na trenirajućim podacima
model.fit(train_data['tag'], train_data['tag'])

# Predvidi oznake na test skupu
predictions = model.predict(test_data['tag'])

# Evaluiraj performanse modela
accuracy = accuracy_score(test_data['tag'], predictions)
print(f'Accuracy: {accuracy}')

# Ispisi detaljan izvještaj o klasifikaciji
print(classification_report(test_data['tag'], predictions))

# Primjer predviđanja novih oznaka za odabrani korisnik
selected_user_tags = ['thriller', 'romantic', 'action']
predicted_tags = model.predict(selected_user_tags)
print(f'Predicted tags for the selected user: {predicted_tags}')


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['action', 'comedy', 'action', 'drama', 'action', 'romance', 'action', 'comedy']
})

# Izgradnja matrice ocjena (1 ako je korisnik dodijelio oznaku, inače 0)
user_movie_matrix = pd.crosstab(user_movie_tags['userId'], user_movie_tags['tag']).clip(upper=1)

# Podijela podataka na trening i test skupove
train_data, test_data = train_test_split(user_movie_matrix, test_size=0.2, random_state=42)

# Računanje sličnosti između korisnika pomoću kosinusne sličnosti
user_similarity = cosine_similarity(train_data)

# Funkcija za predviđanje novih oznaka za korisnika temeljem sličnosti
def predict_tags(user_similarity, user_movie_matrix):
    # Normalizacija sličnosti korisnika
    norm_user_similarity = user_similarity / np.sum(np.abs(user_similarity), axis=1, keepdims=True)

    # Predviđanje novih oznaka za korisnika (koristi sličnost i postojeće oznake)
    predicted_tags = np.dot(norm_user_similarity, user_movie_matrix)

    return predicted_tags

# Predviđanje novih oznaka za test skup korisnika
predicted_tags = predict_tags(user_similarity, test_data.values)

# Prikaz rezultata
predicted_tags_df = pd.DataFrame(predicted_tags, index=test_data.index, columns=test_data.columns)
print("Predicted Tags:")
print(predicted_tags_df)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['action', 'comedy', 'action', 'drama', 'action', 'romance', 'action', 'comedy']
})

# Izgradnja matrice ocjena (1 ako je korisnik dodijelio oznaku, inače 0)
user_movie_matrix = pd.crosstab(user_movie_tags['userId'], user_movie_tags['tag']).clip(upper=1)

# Podijela podataka na trening i test skupove
train_data, test_data = train_test_split(user_movie_matrix, test_size=0.2, random_state=42)

# Računanje sličnosti između filmova pomoću kosinusne sličnosti
movie_similarity = cosine_similarity(train_data.T)

# Funkcija za predviđanje oznaka za odabrani film temeljem sličnosti
def predict_tags(movie_similarity, user_movie_matrix, selected_movie_id):
    # Izdvajanje ocjena za odabrani film
    selected_movie_ratings = user_movie_matrix[selected_movie_id]

    # Računanje ponderirane sličnosti između odabranog filma i ostalih filmova
    weighted_similarity = np.dot(selected_movie_ratings, movie_similarity) / np.sum(np.abs(selected_movie_ratings))

    # Normalizacija predviđenih ocjena u raspon od 0 do 1
    normalized_predictions = (weighted_similarity - np.min(weighted_similarity)) / (np.max(weighted_similarity) - np.min(weighted_similarity))

    return normalized_predictions

# Primjer predviđanja oznaka za odabrani film
selected_movie_id = 105
predicted_tags = predict_tags(movie_similarity, train_data, selected_movie_id)

# Prikaz rezultata
print(f"Predicted tags for Movie ID {selected_movie_id}:")
print(predicted_tags)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['action', 'comedy', 'action', 'drama', 'action', 'romance', 'action', 'comedy']
})

# Izgradnja matrice ocjena (1 ako je korisnik dodijelio oznaku, inače 0)
user_movie_matrix = pd.crosstab(user_movie_tags['userId'], user_movie_tags['tag']).clip(upper=1)

# Podijela podataka na trening i test skupove
train_data, test_data = train_test_split(user_movie_matrix, test_size=0.2, random_state=42)

# Računanje sličnosti između filmova pomoću kosinusne sličnosti
movie_similarity = cosine_similarity(train_data.T)

# Funkcija za predviđanje oznaka za odabrani film temeljem sličnosti
def predict_tags(movie_similarity, user_movie_matrix, selected_movie_id, tag_mapping):
    # Izdvajanje ocjena za odabrani film
    selected_movie_ratings = user_movie_matrix[selected_movie_id]

    # Računanje ponderirane sličnosti između odabranog filma i ostalih filmova
    weighted_similarity = np.dot(selected_movie_ratings, movie_similarity) / np.sum(np.abs(selected_movie_ratings))

    # Normalizacija predviđenih ocjena u raspon od 0 do 1
    normalized_predictions = (weighted_similarity - np.min(weighted_similarity)) / (np.max(weighted_similarity) - np.min(weighted_similarity))

    # Mapiranje normaliziranih predviđenih ocjena na stvarne oznake
    predicted_tags = [tag_mapping[i] for i, value in enumerate(normalized_predictions) if value > 0]

    return predicted_tags

# Mapiranje indeksa oznaka na stvarne oznake
tag_mapping = dict(enumerate(train_data.columns))

# Primjer predviđanja oznaka za odabrani film
selected_movie_id = 105
predicted_tags = predict_tags(movie_similarity, train_data, selected_movie_id, tag_mapping)

# Prikaz rezultata
print(f"Predicted tags for Movie ID {selected_movie_id}:")
print(predicted_tags)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['action', 'comedy', 'action', 'drama', 'action', 'romance', 'action', 'comedy']
})

# Izgradnja matrice ocjena (1 ako je korisnik dodijelio oznaku, inače 0)
user_movie_matrix = pd.crosstab(user_movie_tags['userId'], user_movie_tags['tag']).clip(upper=1)

# Podijela podataka na trening i test skupove
train_data, test_data = train_test_split(user_movie_matrix, test_size=0.2, random_state=42)

# Računanje sličnosti između korisnika pomoću kosinusne sličnosti
user_similarity = cosine_similarity(train_data)

# Funkcija za predviđanje oznaka koje će korisnik koristiti za odabrani film
def predict_user_tags(user_similarity, user_movie_matrix, user_id, movie_id, tag_mapping):
    # Izdvajanje ocjena za odabrani film
    selected_movie_ratings = user_movie_matrix[movie_id]

    # Računanje ponderirane sličnosti između odabranog korisnika i ostalih korisnika
    weighted_similarity = np.dot(selected_movie_ratings, user_similarity[user_id]) / np.sum(np.abs(selected_movie_ratings))

    # Normalizacija predviđenih ocjena u raspon od 0 do 1
    normalized_predictions = (weighted_similarity - np.min(weighted_similarity)) / (np.max(weighted_similarity) - np.min(weighted_similarity))

    # Mapiranje normaliziranih predviđenih ocjena na stvarne oznake
    predicted_tags = [tag_mapping[i] for i, value in enumerate(normalized_predictions) if value > 0]

    return predicted_tags

# Mapiranje indeksa oznaka na stvarne oznake
tag_mapping = dict(enumerate(train_data.columns))

# Primjer predviđanja oznaka koje će korisnik koristiti za odabrani film
selected_user_id = 4
selected_movie_id = 105
predicted_tags = predict_user_tags(user_similarity, train_data, selected_user_id, selected_movie_id, tag_mapping)

# Prikaz rezultata
print(f"Predicted tags for User ID {selected_user_id} and Movie ID {selected_movie_id}:")
print(predicted_tags)
